In [1]:
!pip install elasticsearch-dsl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 1.0 MB/s eta 0:00:00 MB/s eta 0:00:01


In [1]:
import pandas as pd
from elasticsearch.helpers import bulk
from elasticsearch_dsl import Document, Integer, Text, connections

In [2]:
data_path = "data/Sample_trec_1000.tsv"
df = pd.read_csv(data_path, header=None, delimiter=',')
new_column_names = ['0', 'id', 'content']
df.columns = new_column_names
df = df[['id', 'content']]
df.head()

,id,content
0,7130104,This is the definition of RNA along with examp...
1,7130335,Best Answer: The AR designation comes from the...
2,7130336,What does AR really mean? Posted by Quality AR...
3,7130348,Oxycontin is generally prescribed without acet...
4,8001869,STRATEGIC FEDERAL CREDIT UNION Routing number ...


In [31]:
# Define a default Elasticsearch client
connections.create_connection(hosts="http://localhost:9200")

class SampleTrec(Document):
    id = Integer()
    content = Text(analyzer='snowball')

    class Index:
        name = 'sample-trec'
        settings = {
          "number_of_shards": 1,
        }

    def save(self, ** kwargs):
        return super(SampleTrec, self).save(** kwargs)

# create the mappings in elasticsearch
SampleTrec.init()

In [32]:
# Prepare documents for bulk saving
actions = []
for index, row in df.iterrows():
    action = {
        "_op_type": "index",  # operation type
        "_index": "sample-trec",  # index name
        "_id": row['id'],  # document ID
        "_source": {  # document body
            "content": row['content'],
            "id": row["id"]
            # add more fields as needed
        }
    }
    actions.append(action)

# Perform bulk saving
success, _ = bulk(connections.get_connection(), actions)

print(f"Indexed {success} documents successfully.")

Indexed 1001 documents successfully.
